In [ ]:
from functools import *
import random
import timeit

In [ ]:
start = timeit.default_timer()
sleep(3)
end = timeit.default_timer()
print(end-start)

In [ ]:
class AnshelAnshelGoldfeld:
    
    def __init__(self, n, keyLength):
        self.n = n
        self.Bn = BraidGroup(n)
        self.keyLength = keyLength
        
        #Creamos una lista con los índices de las trenzas de la llave pública y sus inversas y a partir
        # de esta creamos la función que creará la llave privada
        index_public_key = list(range(-1*(self.keyLength), self.keyLength+1))
        index_public_key.remove(0)
        self.func = random.choices(index_public_key, k = random.randint(5,10))
        
        self.generatePublicKey()
        self.generatePrivateKey()

        
    #Creamos una lista con los generadores y sus inversos a partir de la cual crearemos las
    #trenzas de la llave pública cuya longitud de palabra estará entre 5 y 10
    def generatePublicKey(self):
        self.publicKey = []
        index_generators = list(range(-1*(self.n)+1, self.n))
        index_generators.remove(0)
        
        for i in range(self.keyLength):
            randomList = random.choices(index_generators, k = random.randint(5,10))
            self.publicKey.append(self.Bn(randomList))

            
    def keyFunction(self, f, l):
        key = self.Bn([1,-1])

        for i in f:
            if i > 0:
                aux = l[i-1]
            else:
                aux = (l[-1*i-1]^-1)
            
            key = key * aux

        return key
            
    def generatePrivateKey(self):
        self.privateKey = self.keyFunction(self.func, self.publicKey)
        
    def messages(self, publicKeyReceiver):
        message = []
        
        for t in publicKeyReceiver:
            message.append(self.lnf(self.privateKey*t*(self.privateKey^-1)))
            
        
        
        return message
    
    def commonKey(self, message, participant):
        if(participant == 0):
            key = self.privateKey*(self.keyFunction(self.func, message)^-1)
        else:
            key = self.keyFunction(self.func, message)*(self.privateKey^-1)

        return self.lnf(key)
    
    def lnf(self,braid):
        normal = self.Bn([1,-1])
        tup = braid.left_normal_form()

        for i in range(len(tup)):
            normal = normal*tup[i]
        
        return normal
        

In [ ]:
def measure(i,j, iterations):
    m = []
    for k in range(iterations):
        start = timeit.default_timer()
        A = AnshelAnshelGoldfeld(i,j)
        B = AnshelAnshelGoldfeld(i,j)
        messageA = A.messages(B.publicKey)
        messageB = B.messages(A.publicKey)
        commonKeyA = A.commonKey(messageB, 0)
        end = timeit.default_timer()
        #commonKeyB = B.commonKey(messageA, 1)
        m.append(end-start)
        if (end-start) > 300:
            break
            
    if len(m) > 1:
        stdes = std(m)
    else:
        stdes = 0;
    
    return len(m), mean(m), stdes, min(m), max(m)

In [ ]:
def test(n_min,m_max,n_s,l_min,l_max,l_s):
    print("size_sample,n,l,time,std,min,max")
    for i in range(n_min,m_max+1,n_s):
        for j in range(l_min,l_max+1,l_s):
            s, t, st, minimo, maximo= measure(i,j,3)
            print(s, ",", i,",",j,",", t,",", st, ",", minimo,",",maximo)
            
            
    

In [ ]:
random.seed(10)

In [ ]:
test(5,25,5,5,10,5)